# LSP - Liskov Substitution Principle

**Obiekty klasy potomnej powinny móc zastąpić obiekty klasy bazowej bez wywoływania błędu w programie.**

Innymi słowy: **Jeżeli masz klasę bazową `Animal`, każda podklasa powinna zachowywać się jak `Animal`.**

## Złe rozwiązanie - łamie LSP

In [5]:
from abc import ABC, abstractmethod

class Animal(ABC):
    @property
    @abstractmethod
    def num_legs(self) -> int:
        pass


class Lion(Animal):
    def __init__(self):
        self.__num_legs = 4
    
    @property
    def num_legs(self):
        return self.__num_legs


class Penguin(Animal):
    def __init__(self):
        self.__num_legs = 2
    
    @property
    def num_legs(self):
        return self.__num_legs


# Klasa łamiąca LSP
class Snake(Animal):
    def __init__(self):
        self.__num_legs = 0  # ← Zwraca None zamiast liczby!
    
    @property
    def num_legs(self):
        return self.__num_legs

In [3]:
# Klasa używająca Animal
class Herd:
    def __init__(self, animal: Animal, total_legs):
        self.animal = animal
        self.total_legs = total_legs
    
    def count_members(self):
        # Zakładamy, że num_legs to liczba całkowita
        return int(self.total_legs / self.animal.num_legs)

In [4]:
# client code without error
lion = Lion()
penguin = Penguin()
snake = Snake()

herds = [
    Herd(lion, 32),
    Herd(penguin, 14),
]

for herd in herds:
    print(f"W stadzie jest {herd.count_members()} członków")


W stadzie jest 8 członków
W stadzie jest 7 członków


In [6]:
# client code with error
lion = Lion()
penguin = Penguin()
snake = Snake()

herds = [
    Herd(lion, 32),
    Herd(penguin, 14),
    Herd(snake, 0)  # ← To spowoduje błąd!
]

for herd in herds:
    print(f"W stadzie jest {herd.count_members()} członków")


W stadzie jest 8 członków
W stadzie jest 7 członków


ZeroDivisionError: division by zero

### Problem:

`Snake.num_legs` zwraca `None`, co powoduje:
```python
int(20 / None)  # TypeError: unsupported operand type(s) for /: 'int' and 'NoneType'
```

**Klasa `Snake` nie może zastąpić `Animal`** - łamie LSP.

---

**Co jeśli zmienimy `None` na `0`?**

```python
class Snake(Animal):
    def __init__(self):
        self.__num_legs = 0  # ← Zmiana na 0
```

**Nadal źle!** Teraz dostaniemy:
```python
int(20 / 0)  # ZeroDivisionError: division by zero
```

## Dobre rozwiązanie - zgodne z LSP

Wężów nie dodajemy do stada (lepszy design).

```python
# Snake nie dziedziczy po Animal, bo nie pasuje do kontraktu
class LeggedAnimal(ABC):
    @property
    @abstractmethod
    def num_legs(self):
        """Zwraca liczbę nóg > 0"""
        pass

class Lion(LeggedAnimal):
    # ...

class Snake:  # ← Osobna hierarchia
    pass
```

## Esencja LSP

### Pytanie:
Czy mogę zastąpić klasę bazową klasą potomną bez psucia działania?

### Złe (łamie LSP):
```python
class Animal:
    def num_legs(self) -> int:  # Kontrakt: zwraca int
        pass

class Snake(Animal):
    def num_legs(self) -> int:
        return None  # ← Zwraca None, łamie kontrakt!
```

### Dobre (zgodne z LSP):
```python
class Animal:
    def num_legs(self) -> int:  # Kontrakt: zwraca int >= 0
        pass

class Snake(Animal):
    def num_legs(self) -> int:
        return 0  # ← Zwraca int, zachowuje kontrakt
```

### Kluczowa obserwacja:

**LSP = zachowanie kontraktu klasy bazowej**

Jeżeli klasa bazowa obiecuje zwracać `int`, podklasa **musi** zwracać `int` (nie `None`, nie `str`).

## Inne przykłady łamania LSP

### Prostokąt vs Kwadrat

Square nie może zastąpić Rectangle - łamie LSP.

In [16]:
class Rectangle:
    def set_width(self, width):
        self.width = width
    
    def set_height(self, height):
        self.height = height


In [17]:
class Square(Rectangle):  # ← Łamie LSP!
    def set_width(self, width):
        self.width = width
        self.height = width  # ← Zmienia również height
    
    def set_height(self, height):
        self.width = height  # ← Zmienia również width
        self.height = height


In [18]:
# Problem:
def test(rect: Rectangle):
    rect.set_width(5)
    rect.set_height(4)
    assert rect.width * rect.height == 20  # OK dla Rectangle


In [19]:
test(Rectangle())  # ok

In [20]:
test(Square())     # Fails! (area = 16, nie 20)

AssertionError: 

## Podsumowanie

### LSP w jednym zdaniu:
**Podklasy muszą zachowywać kontrakt klasy bazowej.**

### Jak rozpoznać łamanie LSP?
- Podklasa zmienia typ zwracany (np. `int` → `None`)
- Podklasa rzuca wyjątki, których bazowa nie rzuca
- Kod działa dla klasy bazowej, ale psuje się dla podklasy

### Jak naprawić?
1. Zachowaj kontrakt (typy zwracane, wyjątki)
3. Wydziel osobną hierarchię (Snake nie dziedziczy po Animal)

### Związek z innymi zasadami:
- **LSP + OCP** - podklasy rozszerzają, ale nie łamią kontraktu
- **LSP + DIP** - interfejsy definiują kontrakt, implementacje go zachowują